<a href="https://colab.research.google.com/github/ykwyuta/timetable/blob/master/timetable_linear_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiita.com/Dason08/items/e1bafb9ddc766d1c8fd0

https://qiita.com/samuelladoco/items/703bf78ea66e8369c455

https://aimap.imi.kyushu-u.ac.jp/wp/wp-content/uploads/2018/01/tutorialKamiyama.pdf

https://kindai.repo.nii.ac.jp/?action=repository_action_common_download&item_id=12802&item_no=1&attribute_id=40&file_no=1

In [1]:
!pip install pulp

     |████████████████████████████████| 13.6MB 1.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


In [0]:
import pulp

In [0]:
problem = pulp.LpProblem("P", pulp.LpMinimize)
x = pulp.LpVariable("x")
y = pulp.LpVariable("y")
# 目的関数
problem += 8*x + 19*y
# 制約
problem += 2*x + 7*y >= 40
problem += 6*x + 3*y >= 50
problem += x >= 0
problem += y >= 0
status = problem.solve()

In [0]:
print(pulp.LpStatus[status])

Optimal


In [0]:
print(pulp.value(problem.objective))

125.00000030000001


In [0]:
print(x.value())
print(y.value())

6.3888889
3.8888889


In [0]:
[a+str(i) for i in range(3) for a in 'abc']

['a0', 'b0', 'c0', 'a1', 'b1', 'c1', 'a2', 'b2', 'c2']

# 時間割最適化
## 変数の定義


In [8]:


problem = pulp.LpProblem("P", pulp.LpMinimize)

subjects = {
    '禁 ': (1, [0, 1, 2, 3, 4, 5]), 
    '現0': (3, [0, 1, 2]), 
    '古0': (2, [0, 1, 2]),
    '漢0': (2, [0, 1, 2]),
    '古1': (2, [3, 4, 5]),
    '漢1': (2, [3, 4, 5]),
    '数0': (5, [0, 1, 2]),
    '英0': (5, [0, 1, 2]),
    '日0': (2, [0, 1, 2]),
    '世0': (2, [3, 4, 5]),
    '化0': (2, [0, 1, 2, 3]),
    '物0': (2, [1, 2, 4, 5]),
    '生0': (2, [0, 3, 4, 5]),
    '数1': (5, [3, 4, 5]),
    '英1': (5, [3, 4, 5]),
    '体0': (1, [0, 1, 2, 3, 4, 5]),
    '美0': (1, [0, 1, 2, 3, 4, 5]),
    '書0': (1, [0, 1, 2, 3, 4, 5]),
    '保0': (1, [0, 1, 2, 3, 4, 5])
}
classes = 5
days = 6
lessons = 5

units = {}

for sk, sv in subjects.items():
  for clazz in range(classes):
    # 担当クラス外はスキップする
    if clazz not in sv[1]:
      continue
    for day in range(days):
      for lesson in range(lessons):
        # 存在の有無（ 0 = 無, 1 = 有）
        units[(sk, clazz, day, lesson)] = pulp.LpVariable("s:{0},c:{1},d:{2},L{3}".format(sk, clazz, day, lesson), 0, 1,pulp.LpBinary)

# 目的関数。今回は制約条件を満たせさえすればいいので意味はない
problem += -units[list(units.keys())[0]], "Objective"
# 制約
# 同時に同じ授業は行われない
for sk, sv in subjects.items():
  if sk == '禁 ':
    continue
  for day in range(days):
    for lesson in range(lessons):
      problem += sum(units[(sk, clazz, day, lesson)]  for clazz in range(classes) if clazz in sv[1]) <= 1

# 1日に同じ授業は1回だけ
for sk, sv in subjects.items():
  for clazz in range(classes):
    if clazz not in sv[1]:
      continue
    for day in range(days):
      if sk == '禁 ':
        continue
      else:
        problem += sum(units[(sk, clazz, day, lesson)] for lesson in range(lessons)) <= 1

# 科目&クラスごとの1週間の最低授業数
for sk, sv in subjects.items():
  for clazz in range(classes):
    if clazz not in sv[1]:
      continue
    if sk == '禁 ':
      problem += sum(units[(sk, clazz, day, lesson)] for day in range(days) for lesson in range(lessons)) == sv[0]
    else:
      problem += sum(units[(sk, clazz, day, lesson)] for day in range(days) for lesson in range(lessons)) >= sv[0]

# 1日の授業数は決まっている
for clazz in range(classes):
  for day in range(days):
    for lesson in range(lessons):
      problem += sum(units[(sk, clazz, day, lesson)]  for sk, sv in subjects.items() if clazz in sv[1]) == 1

# 授業が実施できない枠を設定する        
for clazz in range(classes):
  problem += units[('禁 ', clazz, days - 1, lessons - 1)] == 1
  
status = problem.solve()
print(pulp.LpStatus[status])

Optimal


In [0]:
# problem

In [10]:
for clazz in range(classes):
  for day in range(days):
    sub = []
    for lesson in range(lessons):
      for sk, sv in subjects.items():
          if clazz not in sv[1]:
            continue
          if units[(sk, clazz, day, lesson)].value() > 0:
            sub.append(str(lesson) + ':' + sk)
    print("c:{0},d:{1}_".format(clazz, day) + ",".join(sub))

c:0,d:0_0:保0,1:古0,2:体0,3:英0,4:日0
c:0,d:1_0:数0,1:英0,2:漢0,3:古0,4:体0
c:0,d:2_0:化0,1:英0,2:数0,3:生0,4:現0
c:0,d:3_0:書0,1:漢0,2:英0,3:数0,4:現0
c:0,d:4_0:数0,1:化0,2:現0,3:英0,4:日0
c:0,d:5_0:数0,1:書0,2:美0,3:生0,4:禁 
c:1,d:0_0:体0,1:書0,2:漢0,3:古0,4:物0
c:1,d:1_0:日0,1:物0,2:古0,3:数0,4:英0
c:1,d:2_0:英0,1:美0,2:漢0,3:数0,4:化0
c:1,d:3_0:化0,1:数0,2:現0,3:英0,4:日0
c:1,d:4_0:現0,1:保0,2:数0,3:日0,4:英0
c:1,d:5_0:現0,1:保0,2:数0,3:英0,4:禁 
c:2,d:0_0:英0,1:保0,2:物0,3:化0,4:書0
c:2,d:1_0:漢0,1:古0,2:数0,3:英0,4:物0
c:2,d:2_0:数0,1:化0,2:現0,3:保0,4:英0
c:2,d:3_0:数0,1:体0,2:日0,3:現0,4:漢0
c:2,d:4_0:日0,1:数0,2:英0,3:現0,4:美0
c:2,d:5_0:化0,1:数0,2:英0,3:古0,4:禁 
c:3,d:0_0:数1,1:古1,2:生0,3:英1,4:化0
c:3,d:1_0:漢1,1:生0,2:世0,3:数1,4:英1
c:3,d:2_0:世0,1:体0,2:数1,3:英1,4:美0
c:3,d:3_0:体0,1:数1,2:保0,3:英1,4:世0
c:3,d:4_0:英1,1:書0,2:化0,3:古1,4:生0
c:3,d:5_0:書0,1:漢1,2:生0,3:数1,4:禁 
c:4,d:0_0:英1,1:数1,2:古1,3:物0,4:保0
c:4,d:1_0:物0,1:数1,2:古1,3:英1,4:世0
c:4,d:2_0:保0,1:英1,2:世0,3:物0,4:数1
c:4,d:3_0:漢1,1:英1,2:美0,3:体0,4:生0
c:4,d:4_0:生0,1:古1,2:英1,3:数1,4:漢1
c:4,d:5_0:漢1,1:数1,2:書0,3:体0,4:禁 
